In [13]:
import numpy as np 
from typing import Optional 
import logging
logging.basicConfig(level=logging.INFO)
def should_I_buy_infinite_tickets_v2(
        ticket_price : int = 2, 
        cash : Optional[int] = 625_000_000,
        annunity : Optional[int] = None,
        effective_tax_rate : float = 0.37, # https://www.usatoday.com/story/graphics/2023/01/10/mega-millions-jackpot-lottery-taxes/11001363002/
) -> bool: 
    """ Version 2 considers the non jackpot prizes and their probabilities in the calculation."""

    # TODO: make a graph of jackpot vs break even ticket prize
    # TODO: consider inflation in annunity mode
    if (annunity is None) == (cash is None): 
        raise ValueError(f"Exactly one of `annunity` and `cash` must be None. {annunity=}. {cash=} ")
    if annunity is not None:
        jackpot = annunity 
        logging.info(f"Annunity mode selected. Total annunity amount: ${format(annunity, ',')}")
        logging.warning(f"We assume the effective tax rate ({effective_tax_rate}) is the same  over the lifetime of the annunity. We do not account for inflation")
    else: 
        logging.info(f"Cash jackpot mode selected. Jackpot amount: ${format(cash, ',')}")
        jackpot = cash


    logging.info(f"ticket_price=${ticket_price}")    
    logging.info(f"{effective_tax_rate=}")


    # Prizes and probabilities from: https://www.megamillions.com/How-to-Play.aspx
    prizes = np.array([jackpot, 1_000_000, 10_000, 500, 200, 10, 10, 4, 2])
    probabilities = np.array([1/302_575_350, 1/12_607_306, 1/931_001, 1/38_792, 1/14_547, 1/606, 1/693, 1/89, 1/37])

    prizes_post_tax = prizes * (1 - effective_tax_rate) # For now, naively just apply the hightest tax rate to each price. Probably not a big issue since expected value is dominated by the largest prizes which get the highest tax rate.


    scaled_prizes = probabilities * prizes_post_tax
    expected_value_post_tax = sum(scaled_prizes)
    logging.info(f"expected_value_post_tax=${format(round(expected_value_post_tax, 2), ',')}")

    # break even point is the point where the jackpot amount is large enough such that the expected value of the ticket equals the ticket prize. We therefore solve the following equation for 'break_even_jackpot'
    # ticket_price = (1 - effective_tax_rate) * break_even_jackpot * probability_of_winning_jackpot + expected_value_for_non_jackpot_prizes 
    # break_even_jackpot = (ticket_price - expected value for non jackpot prizes) /  ((1 - effective_tax_rate) * probability_of_winning_jackpot)
    break_even_jackpot = (ticket_price - sum(scaled_prizes[1:])) / ((1 - effective_tax_rate) * probabilities[0])
    logging.info(f"break_event_point=${format(break_even_jackpot, ',')}")

    if expected_value_post_tax >= ticket_price:
        logging.info(f"Expected value of the ticket post tax (${round(expected_value_post_tax, 2)}) is greater than or equal to ticket price (${ticket_price}). You should buy infinite tickets.")
        return True 
    else: 
        logging.info(f"Expected value of the ticket post tax (${round(expected_value_post_tax, 2)}) is less than the ticket price ({ticket_price}). Wait until the {'annuity total' if annunity is not None else 'jackpot price'} reaches ${format(break_even_jackpot, ',')} or more.")

should_I_buy_infinite_tickets_v2()

INFO:root:Cash jackpot mode selected. Jackpot amount: $625,000,000
INFO:root:ticket_price=$2
INFO:root:effective_tax_rate=0.37
INFO:root:expected_value_post_tax=$1.46
INFO:root:break_event_point=$885,933,250.1571822
INFO:root:Expected value of the ticket post tax ($1.46) is less than the ticket price (2). Wait until the jackpot price reaches $885,933,250.1571822 or more.
